 ![CellphoneDB Logo](https://www.cellphonedb.org/images/cellphonedb_logo_33.png) | CellphoneDB is a publicly available repository of curated receptors, ligands and their interactions. ||
 :------------- | :------------- | :-------------

CellphoneDB includes subunit architecture for both ligands and receptors, representing heteromeric complexes accurately. This is crucial, as cell-cell communication relies on multi-subunit protein complexes that go beyond the binary representation used in most databases and studies.

CellPhoneDB integrates existing datasets that pertain to cellular communication and new manually reviewed information. CellPhoneDB utilises information from the following data bases: [UniProt](https://www.uniprot.org/), [Ensembl](https://www.ensembl.org/), [PDB](https://www.ebi.ac.uk/pdbe/), [the IMEx consortium](https://www.imexconsortium.org/) and [IUPHAR](https://www.guidetopharmacology.org/).

CellPhoneDB can be used to search for a particular ligand/receptor or interrogate your own single-cell transcriptomics data.



## Install CellphoneDB package

In [ ]:
%%capture
pip install --force-reinstall "git+https://github.com/ventolab/CellphoneDB.git@bare-essentials"

## List CellphoneDB data releases

In [ ]:
from IPython.display import HTML, display
from cellphonedb.utils import db_releases_utils
display(HTML(db_releases_utils.get_remote_database_versions_html()['db_releases_html_table']))

## Set CellphoneDB version and local directories for the database and user data

In [ ]:
import os
# The default version of CellphoneDB data is the latest one, but you can change it to a previous version 
# at any point in this notebook (by re-setting the value of cpdb_version variable). 
# Please note that the format of the database from version v4.1.0 is incompatible with that of previous 
# versions, hence the lowest version number you may choose in this notebook is v4.1.0
cpdb_version = "v4.1.4"
# N.B. At the very least, please replace <your_user_id> with your user id
cpdb_dir = os.path.join("/Users/<your_user_id>/.cpdb/releases", cpdb_version)

## Download CellphoneDB database from https://github.com/ventolab/cellphonedb-data/

In [ ]:
import os
from cellphonedb.utils import db_utils
db_utils.download_database(cpdb_dir, cpdb_version)
# If you generated your own CellphoneDB database file, please replace the default path below to the your file's path
cpdb_file_path = os.path.join(cpdb_dir, "cellphonedb.zip")

## Search CellphoneDB Interactions
#### Search CellphoneDB interactions by (a comma- or space-separated list of): 
* Ensembl ID (e.g. ENSG00000165029), 
* Gene name (e.g. ABCA1), 
* UniProt ID (e.g. KLRG2_HUMAN), 
* UniProt Accession (e.g. A4D1S0) or 
* Complex name (e.g. 12oxoLeukotrieneB4_byPTGR1)

In [ ]:
import os
from cellphonedb.utils import file_utils, search_utils
from IPython.display import HTML, display
# Search CellphoneDB interactions by (a comma- or space-separated list of):
# Ensembl ID (e.g. ENSG00000165029), Gene name (e.g. ABCA1), UniProt ID (e.g. KLRG2_HUMAN), 
# UniProt Accession (e.g. A4D1S0) or Complex name (e.g. 12oxoLeukotrieneB4_byPTGR1)
(results, complex_name2proteins_text) = search_utils.search('ENSG00000134780,integrin_a10b1_complex', cpdb_file_path)
# Display results in a html table
# Note: Mouse over complex names to see constituent proteins
display(HTML(search_utils.get_html_table(results, complex_name2proteins_text)))

## Run Basic Analysis

In [ ]:
# Please populate the following variables before executing the analysis
meta_file_path = None
counts_file_path = None
output_path = None
# Execute basic analysis
from cellphonedb.src.core.methods import cpdb_analysis_method
means, significant_means, deconvoluted = cpdb_analysis_method.call(
    cpdb_file_path = cpdb_file_path, 
    meta_file_path = meta_file_path, 
    counts_file_path = counts_file_path,
    counts_data = 'ensembl',
    output_path = output_path,
    microenvs_file_path = None,
    separator = "|",
    threshold = 0.1,
    result_precision = 3,
    debug = False,
    output_suffix = None)
# print(means.info)
# print(significant_means.info)
# print(deconvoluted.info)

## Run Statistical Analysis

In [ ]:
# Please populate the following variables before executing the analysis
meta_file_path = None
counts_file_path = None
output_path = None
# Execute statistical analysis
from cellphonedb.src.core.methods import cpdb_statistical_analysis_method
deconvoluted, means, pvalues, significant_means = \
    cpdb_statistical_analysis_method.call(
        cpdb_file_path = cpdb_file_path, 
        meta_file_path = meta_file_path, 
        counts_file_path = counts_file_path,
        counts_data = 'ensembl',
        output_path = output_path,
        microenvs_file_path = None,
        iterations = 1000,
        threshold = 0.1,
        threads = 4,
        debug_seed = -1,
        result_precision = 3,
        pvalue = 0.05,
        subsampling = False,
        subsampling_log = False,
        subsampling_num_pc = 100,
        subsampling_num_cells = None,
        separator = '|',
        debug = False,
        output_suffix = None)
# print(deconvoluted.info)
# print(means.info)
# print(pvalues.info)
# print(significant_means.info)


## Run Differential Analysis

In [ ]:
# Please populate the following variables before executing the analysis
meta_file_path = None
counts_file_path = None
degs_file_path = None
output_path = None
# Execute differential analysis
from cellphonedb.src.core.methods import cpdb_degs_analysis_method
deconvoluted, means, relevant_interactions, significant_means = \
    cpdb_degs_analysis_method.call(
        cpdb_file_path = cpdb_file_path, 
        meta_file_path = meta_file_path, 
        counts_file_path = counts_file_path,
        degs_file_path = degs_file_path,
        counts_data = 'ensembl',
        microenvs_file_path=None,
        threshold = 0.1,
        debug_seed = -1,
        result_precision = 3,
        separator = '|',
        debug = False,
        output_path = output_path,
        output_suffix = None)
# print(deconvoluted.info)
# print(means.info)
# print(relevant_interactions.info)
# print(significant_means.info)

## Plot Statistical Analysis results

In [ ]:
import warnings
warnings.filterwarnings('ignore')
from ktplotspy.plot import plot_cpdb, plot_cpdb_heatmap
from cellphonedb.utils import file_utils
import os

meta_fp='test_meta.txt'
# counts_fn='test_counts.txt'
counts_fp='test.h5ad'
# Create AnnData object with obs set to a DataFrame containing data from meta_fn
adata = file_utils.get_counts_meta_adata(counts_fp, meta_fp)

# Exmaple dot plot
g1 = plot_cpdb(
        adata=adata,
        cell_type1="Myeloid",
        # '.' means any cell type
        cell_type2=".",
        means=means,
        pvals=pvalues,
        celltype_key="cell_type",
        genes=["FN1", "integrin-a5b1-complex","COLEC12"],
        title="Example dot plot"
    )

# Example heatmap
g2 = plot_cpdb_heatmap(
        adata=adata,
        pvals=pvalues,
        celltype_key="cell_type",
        log1p_transform=True,
        title="Example heatmap"
    )
g1, g2
